In [1]:
import tensorflow as tf
import numpy as np

In [2]:

# Load CSV using Pandas
import pandas
filename = './data/training_input.csv'
names = ['id','entryUrl','productLink','odds','winners','startTime','endTime','price','hard_endTime','hasVideo']
df = pandas.read_csv(filename, names=names)
print(df.shape)

(80056, 10)


C:\Users\dan.KWINS\AppData\Local\Continuum\anaconda3\envs\ML\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,4,5,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df.drop(['id', 'entryUrl', 'productLink'], axis=1)
df = df.drop([0], axis=0)


In [4]:
import pandas as pd
df.winners = df.winners.astype(int)
df.startTime = df.startTime.astype(int)
df.endTime = df.endTime.astype(int)
df.price = df.price.str.replace('$','').astype(float)
df.dtypes
df.shape

(80055, 7)

In [5]:
df = df[df.odds.str.contains('varies') != True]
print(df.shape)
df = df.dropna()


(69780, 7)


In [6]:
df.shape


(62445, 7)

In [7]:
df.hard_endTime = pd.to_numeric(df.hard_endTime)
df.hard_endTime = df.hard_endTime.astype(int)


In [8]:
df.hasVideo = pd.to_numeric(df.hasVideo)
df.hasVideo = df.hasVideo.astype(str)
df.odds = pd.to_numeric(df.odds)
df.odds = df.odds.astype(int)

In [9]:
df.dtypes

odds              int32
winners           int32
startTime         int32
endTime           int32
price           float64
hard_endTime      int32
hasVideo         object
dtype: object

In [10]:
df['startTimeDT'] = pd.to_datetime(df['startTime'],unit='s')
df['endTimeDT'] = pd.to_datetime(df['hard_endTime'],unit='s')
df['startTime'] = df['startTime'] - (7*60*60)
df['duration'] = df.endTime - df.startTime

In [11]:
df.describe()

,odds,winners,startTime,endTime,price,hard_endTime,duration
count,62445.000000,62445.000000,6.244500e+04,6.244500e+04,62445.00000,6.244500e+04,6.244500e+04
mean,1585.248843,3.953111,1.528539e+09,1.528853e+09,21.31873,1.529190e+09,3.142785e+05
std,1650.905471,6.517710,3.298784e+06,3.285758e+06,21.41149,3.338298e+06,2.911422e+05
min,1.000000,1.000000,1.517436e+09,1.517558e+09,0.00000,1.517558e+09,-1.925100e+06
25%,300.000000,1.000000,1.525668e+09,1.526045e+09,9.99000,1.526368e+09,5.508000e+04
50%,1300.000000,2.000000,1.528864e+09,1.529219e+09,16.61000,1.529564e+09,1.440000e+05
75%,2000.000000,3.000000,1.531566e+09,1.531757e+09,25.99000,1.532243e+09,6.612600e+05
max,63000.000000,50.000000,1.534329e+09,1.534316e+09,649.00000,1.534316e+09,2.712960e+06


In [12]:
df = df[df['duration'] > 0]
df['startDay'] = df['startTimeDT'].dt.dayofweek.astype(str)
df['startDayOfMonth'] = df['startTimeDT'].dt.day.astype(str)
df['startMonth'] = df['startTimeDT'].map(lambda x: x.month).astype(str)
df['startHour'] = df['startTimeDT'].map(lambda x: x.hour).astype(str)
df['endDay'] = df['endTimeDT'].dt.dayofweek.astype(str)
df['endMonth'] = df['endTimeDT'].map(lambda x: x.month).astype(str)
df['endHour'] = df['endTimeDT'].map(lambda x: x.hour).astype(str)
df['endDayOfMonth'] = df['endTimeDT'].dt.day.astype(str)


In [13]:
x = df[['odds','winners','price','hasVideo',
        'startDay','startHour','startDayOfMonth','endDay',
        'endHour','endDayOfMonth']] 
y = df['duration']
df.head()

,odds,winners,startTime,endTime,price,hard_endTime,hasVideo,startTimeDT,endTimeDT,duration,startDay,startDayOfMonth,startMonth,startHour,endDay,endMonth,endHour,endDayOfMonth
5,300,2,1533633240,1534316340,4.99,1534316340,0,2018-08-07 16:14:00,2018-08-15 06:59:00,683100,1,7,8,16,2,8,6,15
6,2200,1,1533632940,1533704820,32.99,1534316340,0,2018-08-07 16:09:00,2018-08-15 06:59:00,71880,1,7,8,16,2,8,6,15
7,1900,1,1533632760,1534316340,150.00,1534316340,0,2018-08-07 16:06:00,2018-08-15 06:59:00,683580,1,7,8,16,2,8,6,15
8,800,5,1533632760,1534316340,29.22,1534316340,1,2018-08-07 16:06:00,2018-08-15 06:59:00,683580,1,7,8,16,2,8,6,15
9,2500,1,1533632700,1533684900,39.99,1534316340,0,2018-08-07 16:05:00,2018-08-15 06:59:00,52200,1,7,8,16,2,8,6,15


In [14]:
x.columns

Index(['odds', 'winners', 'price', 'hasVideo', 'startDay', 'startHour',
       'startDayOfMonth', 'endDay', 'endHour', 'endDayOfMonth'],
      dtype='object')

In [15]:
cols_to_norm = ['odds', 'winners', 'price']
x[cols_to_norm] = x[cols_to_norm].apply(lambda x: (x - x.min())/(x.max() - x.min()))
x.head()

C:\Users\dan.KWINS\AppData\Local\Continuum\anaconda3\envs\ML\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,odds,winners,price,hasVideo,startDay,startHour,startDayOfMonth,endDay,endHour,endDayOfMonth
5,0.004746,0.020408,0.007689,0,1,16,7,2,6,15
6,0.034905,0.000000,0.050832,0,1,16,7,2,6,15
7,0.030143,0.000000,0.231125,0,1,16,7,2,6,15
8,0.012683,0.081633,0.045023,1,1,16,7,2,6,15
9,0.039667,0.000000,0.061618,0,1,16,7,2,6,15


In [16]:
x.columns

odds = tf.feature_column.numeric_column('odds')
winners = tf.feature_column.numeric_column('winners')
price = tf.feature_column.numeric_column('price')
hasVideo =  tf.feature_column.categorical_column_with_vocabulary_list('hasVideo',['0','1'])
startDay = tf.feature_column.categorical_column_with_vocabulary_list('startDay',['0','1','2','3','4','5','6'])
startHour = tf.feature_column.categorical_column_with_vocabulary_list('startHour',[str(i) for i in list(range(0,23))])
startDayOfMonth = tf.feature_column.categorical_column_with_vocabulary_list('startDayOfMonth',[str(i) for i in list(range(1,32))])
endDay = tf.feature_column.categorical_column_with_vocabulary_list('endDay',['0','1','2','3','4','5','6'])
endHour = tf.feature_column.categorical_column_with_vocabulary_list('endHour',[str(i) for i in list(range(0,23))])
endDayOfMonth = tf.feature_column.categorical_column_with_vocabulary_list('endDayOfMonth',[str(i) for i in list(range(1,32))])


In [17]:
feat_cols = [odds, winners, price, hasVideo, startDay, startHour, startDayOfMonth, endDay, endHour, endDayOfMonth]

In [18]:
#TRAIN TEST SPLIT

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)

In [21]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs=1000, shuffle=True)

In [22]:
# DNN

In [23]:
emb_hasVideo = tf.feature_column.embedding_column(hasVideo, dimension=2)
emb_startDay = tf.feature_column.embedding_column(startDay, dimension=7)
emb_startHour = tf.feature_column.embedding_column(startHour, dimension=24)
emb_startDayOfMonth = tf.feature_column.embedding_column(startDayOfMonth, dimension=31)
emb_endDay = tf.feature_column.embedding_column(endDay, dimension=7)
emb_endHour = tf.feature_column.embedding_column(endHour, dimension=24)
emb_endDayOfMonth = tf.feature_column.embedding_column(endDayOfMonth, dimension=31)

In [24]:
feat_cols = [odds, winners, price, emb_hasVideo, emb_startDay, emb_startHour, emb_startDayOfMonth, emb_endDay, emb_endHour, emb_endDayOfMonth]

In [25]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train,y_train,batch_size=100,num_epochs=8000,shuffle=True)

In [30]:
dnn_model = tf.estimator.DNNRegressor([10,20,80,160,80,20,10], feature_columns=feat_cols, model_dir='./models/v2')


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/v2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020955758BA8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
dnn_model.train(input_fn=input_func,steps=1000000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/v2\model.ckpt.
INFO:tensorflow:loss = 19988455000000.0, step = 0
INFO:tensorflow:global_step/sec: 82.4697
INFO:tensorflow:loss = 7489319300000.0, step = 100 (1.212 sec)
INFO:tensorflow:global_step/sec: 119.793
INFO:tensorflow:loss = 6932818000000.0, step = 200 (0.835 sec)
INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 6789758000000.0, step = 300 (0.824 sec)
INFO:tensorflow:global_step/sec: 120.658
INFO:tensorflow:loss = 13228851000000.0, step = 400 (0.828 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 7942607700000.0, step = 500 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 7009188000000.0, step = 600 (0.822 sec)
INFO:tensorflow:gl

INFO:tensorflow:global_step/sec: 119.794
INFO:tensorflow:loss = 6840366700000.0, step = 7600 (0.835 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 7696100600000.0, step = 7700 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 6524517000000.0, step = 7800 (0.824 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 5816315300000.0, step = 7900 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 6769237600000.0, step = 8000 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 6595877500000.0, step = 8100 (0.823 sec)
INFO:tensorflow:global_step/sec: 122.726
INFO:tensorflow:loss = 5679324600000.0, step = 8200 (0.815 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 5599690000000.0, step = 8300 (0.820 sec)
INFO:tensorflow:global_step/sec: 122.726
INFO:tensorflow:loss = 6802288700000.0, step = 8400 (0.815 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflo

INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 5731325600000.0, step = 15400 (0.821 sec)
INFO:tensorflow:global_step/sec: 120.803
INFO:tensorflow:loss = 6396718400000.0, step = 15500 (0.828 sec)
INFO:tensorflow:global_step/sec: 121.242
INFO:tensorflow:loss = 7117758000000.0, step = 15600 (0.825 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 5637660000000.0, step = 15700 (0.824 sec)
INFO:tensorflow:global_step/sec: 120.496
INFO:tensorflow:loss = 6547463700000.0, step = 15800 (0.830 sec)
INFO:tensorflow:global_step/sec: 120.95
INFO:tensorflow:loss = 11221263000000.0, step = 15900 (0.827 sec)
INFO:tensorflow:global_step/sec: 121.684
INFO:tensorflow:loss = 6159249400000.0, step = 16000 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 7655021600000.0, step = 16100 (0.824 sec)
INFO:tensorflow:global_step/sec: 118.379
INFO:tensorflow:loss = 6839349500000.0, step = 16200 (0.845 sec)
INFO:tensorflow:global_step/sec: 118.941
INFO:

INFO:tensorflow:global_step/sec: 117.135
INFO:tensorflow:loss = 4588483600000.0, step = 23200 (0.854 sec)
INFO:tensorflow:global_step/sec: 118.413
INFO:tensorflow:loss = 6963567500000.0, step = 23300 (0.844 sec)
INFO:tensorflow:global_step/sec: 117.342
INFO:tensorflow:loss = 5266831000000.0, step = 23400 (0.853 sec)
INFO:tensorflow:global_step/sec: 117.547
INFO:tensorflow:loss = 6465401700000.0, step = 23500 (0.851 sec)
INFO:tensorflow:global_step/sec: 115.648
INFO:tensorflow:loss = 5331740500000.0, step = 23600 (0.864 sec)
INFO:tensorflow:global_step/sec: 113.553
INFO:tensorflow:loss = 7052123400000.0, step = 23700 (0.881 sec)
INFO:tensorflow:global_step/sec: 116.59
INFO:tensorflow:loss = 6086695300000.0, step = 23800 (0.858 sec)
INFO:tensorflow:global_step/sec: 118.941
INFO:tensorflow:loss = 6297194000000.0, step = 23900 (0.841 sec)
INFO:tensorflow:global_step/sec: 116.862
INFO:tensorflow:loss = 5307983000000.0, step = 24000 (0.857 sec)
INFO:tensorflow:global_step/sec: 118.379
INFO:t

INFO:tensorflow:loss = 6403073000000.0, step = 30900 (0.819 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 9751156000000.0, step = 31000 (0.824 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 5824508300000.0, step = 31100 (0.825 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 5809282000000.0, step = 31200 (0.820 sec)
INFO:tensorflow:global_step/sec: 122.277
INFO:tensorflow:loss = 6364976000000.0, step = 31300 (0.819 sec)
INFO:tensorflow:global_step/sec: 122.277
INFO:tensorflow:loss = 5654623300000.0, step = 31400 (0.818 sec)
INFO:tensorflow:global_step/sec: 118.941
INFO:tensorflow:loss = 14012963000000.0, step = 31500 (0.841 sec)
INFO:tensorflow:global_step/sec: 118.379
INFO:tensorflow:loss = 6170509000000.0, step = 31600 (0.844 sec)
INFO:tensorflow:global_step/sec: 118.778
INFO:tensorflow:loss = 6070516000000.0, step = 31700 (0.842 sec)
INFO:tensorflow:global_step/sec: 108.48
INFO:tensorflow:loss = 6687567000000.0, step =

...

INFO:tensorflow:loss = 3756288400000.0, step = 92900 (0.835 sec)
INFO:tensorflow:global_step/sec: 121.535
INFO:tensorflow:loss = 5064999000000.0, step = 93000 (0.822 sec)
INFO:tensorflow:global_step/sec: 120.08
INFO:tensorflow:loss = 5631790500000.0, step = 93100 (0.834 sec)
INFO:tensorflow:global_step/sec: 120.08
INFO:tensorflow:loss = 6319112000000.0, step = 93200 (0.832 sec)
INFO:tensorflow:global_step/sec: 120.949
INFO:tensorflow:loss = 4828869700000.0, step = 93300 (0.827 sec)
INFO:tensorflow:global_step/sec: 121.095
INFO:tensorflow:loss = 5526528700000.0, step = 93400 (0.827 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 5193438000000.0, step = 93500 (0.820 sec)
INFO:tensorflow:global_step/sec: 119.65
INFO:tensorflow:loss = 5862742600000.0, step = 93600 (0.836 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 4858591600000.0, step = 93700 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 6480434000000.0, step = 93

INFO:tensorflow:global_step/sec: 112.913
INFO:tensorflow:loss = 5993112000000.0, step = 100700 (0.886 sec)
INFO:tensorflow:global_step/sec: 119.082
INFO:tensorflow:loss = 4301270300000.0, step = 100800 (0.840 sec)
INFO:tensorflow:global_step/sec: 119.65
INFO:tensorflow:loss = 5076507700000.0, step = 100900 (0.836 sec)
INFO:tensorflow:global_step/sec: 118.1
INFO:tensorflow:loss = 5740299000000.0, step = 101000 (0.847 sec)
INFO:tensorflow:global_step/sec: 119.65
INFO:tensorflow:loss = 5262335000000.0, step = 101100 (0.837 sec)
INFO:tensorflow:global_step/sec: 120.08
INFO:tensorflow:loss = 5743023000000.0, step = 101200 (0.832 sec)
INFO:tensorflow:global_step/sec: 120.804
INFO:tensorflow:loss = 5238452000000.0, step = 101300 (0.828 sec)
INFO:tensorflow:global_step/sec: 120.949
INFO:tensorflow:loss = 6432538000000.0, step = 101400 (0.827 sec)
INFO:tensorflow:global_step/sec: 118.8
INFO:tensorflow:loss = 5404185600000.0, step = 101500 (0.843 sec)
INFO:tensorflow:global_step/sec: 120.804
INF

INFO:tensorflow:global_step/sec: 120.658
INFO:tensorflow:loss = 5144852000000.0, step = 108400 (0.828 sec)
INFO:tensorflow:global_step/sec: 120.804
INFO:tensorflow:loss = 5963000000000.0, step = 108500 (0.829 sec)
INFO:tensorflow:global_step/sec: 121.242
INFO:tensorflow:loss = 5514101500000.0, step = 108600 (0.824 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 7724559500000.0, step = 108700 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 5578747000000.0, step = 108800 (0.821 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 5038120400000.0, step = 108900 (0.822 sec)
INFO:tensorflow:global_step/sec: 120.949
INFO:tensorflow:loss = 5149725000000.0, step = 109000 (0.826 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 4414280600000.0, step = 109100 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 4989231600000.0, step = 109200 (0.821 sec)
INFO:tensorflow:global_step/sec: 121.

INFO:tensorflow:global_step/sec: 110.766
INFO:tensorflow:loss = 4752899300000.0, step = 116100 (0.903 sec)
INFO:tensorflow:global_step/sec: 118.519
INFO:tensorflow:loss = 6577820500000.0, step = 116200 (0.844 sec)
INFO:tensorflow:global_step/sec: 119.351
INFO:tensorflow:loss = 5542521600000.0, step = 116300 (0.838 sec)
INFO:tensorflow:global_step/sec: 119.65
INFO:tensorflow:loss = 5650624500000.0, step = 116400 (0.836 sec)
INFO:tensorflow:global_step/sec: 120.949
INFO:tensorflow:loss = 6231322300000.0, step = 116500 (0.828 sec)
INFO:tensorflow:global_step/sec: 121.242
INFO:tensorflow:loss = 4467729000000.0, step = 116600 (0.824 sec)
INFO:tensorflow:global_step/sec: 109.942
INFO:tensorflow:loss = 5770064000000.0, step = 116700 (0.911 sec)
INFO:tensorflow:global_step/sec: 114.591
INFO:tensorflow:loss = 6442505000000.0, step = 116800 (0.872 sec)
INFO:tensorflow:global_step/sec: 115.515
INFO:tensorflow:loss = 6441123400000.0, step = 116900 (0.867 sec)
INFO:tensorflow:global_step/sec: 113.4

INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 5471878500000.0, step = 123800 (0.822 sec)
INFO:tensorflow:global_step/sec: 122.128
INFO:tensorflow:loss = 6399335000000.0, step = 123900 (0.819 sec)
INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 5499198600000.0, step = 124000 (0.824 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 5051269500000.0, step = 124100 (0.819 sec)
INFO:tensorflow:global_step/sec: 121.242
INFO:tensorflow:loss = 5870862700000.0, step = 124200 (0.825 sec)
INFO:tensorflow:global_step/sec: 121.242
INFO:tensorflow:loss = 5075404000000.0, step = 124300 (0.825 sec)
INFO:tensorflow:global_step/sec: 122.277
INFO:tensorflow:loss = 9985650000000.0, step = 124400 (0.819 sec)
INFO:tensorflow:global_step/sec: 119.937
INFO:tensorflow:loss = 5938792000000.0, step = 124500 (0.834 sec)
INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 5080823600000.0, step = 124600 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.

In [ ]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)


In [ ]:
dnn_model.evaluate(eval_input_func)

In [ ]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=10, num_epochs=1, shuffle=False)
predictions = dnn_model.predict(pred_input_func)

In [ ]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])



In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,final_preds)**0.5

In [ ]:
final_preds
